In [1]:
import os
import sys
import torch
import torch.nn.functional as F
import torchaudio
import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
import numpy as np
from tqdm import tqdm
import logging
from hyperpyyaml import load_hyperpyyaml
import csv
device = torch.device('cuda')
logger = logging.getLogger(__name__)

# from mir_eval.separation import bss_eval_sources
from speechbrain.dataio.dataio import read_audio
from fast_bss_eval import bss_eval_sources

c:\Users\Arian\.conda\envs\inference\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
def read_all_audio(path):
   audio_ids = []
   out_target = {}
   out_est = {}
   out_mix = []
   for filename in os.listdir(path):
      filepath = os.path.join(path, filename)
      item_num = filename.split('_')[0]
      audio = read_audio(filepath).unsqueeze(0)
      if "hat" in filename:
         if  item_num in out_est:
            out_est[item_num].append(audio)
         else: 
            out_est[item_num]  = [audio]
      elif "mix" not in filename:
         if  item_num in out_est:
            out_target[item_num].append(audio)
         else: 
            out_target[item_num]  = [audio]
      elif "mix" in filename:
         audio_ids.append(item_num.replace('item',''))
         out_mix.append(audio)
   out_target = list(out_target.values())
   out_est = list(out_est.values())
      
   return audio_ids, out_target, out_est, out_mix

audio_ids,out_target, out_est, out_mix = read_all_audio('results/audio_results')

In [3]:
class Separation(sb.Brain):
   def compute_objectives(self, predictions, targets):
        """Computes the si-snr loss"""
        return self.hparams.loss(targets, predictions)
     
   def get_metrics(self,audio_ids, targets, preds, mixtures):

        # Create folders where to store audio
      save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init

      all_sdrs = []
      all_sdrs_i = []
      all_sisnrs = []
      all_sisnrs_i = []
      csv_columns = ["audio_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

        

      with open(save_file, "w") as results_csv:
         writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
         writer.writeheader()
            
         for audio_id, target, pred,mixture_ in tqdm(zip(audio_ids,targets, preds,mixtures), total=len(targets)):
   
            # Compute SI-SNR
            target = torch.cat(
            [target[i].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)
            
            pred = torch.cat(
            [pred[i].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)
  
            sisnr = self.compute_objectives(pred, target)
            # COmpute SI-SNR Improvement
            mixture_signal = torch.stack(
               [mixture_] * self.hparams.num_spks, dim=-1
            )
            mixture_signal = mixture_signal.to(target.device)
            sisnr_baseline = self.compute_objectives(
               mixture_signal, target
            )
            sisnr_i = sisnr.cpu().numpy() - sisnr_baseline.cpu().numpy()
  # Compute SDR
            sdr, _, _, _ = bss_eval_sources(
               target[0].t(),
               pred[0].t(),
            )

            sdr_baseline, _, _, _ = bss_eval_sources(
               target[0].t(),
               mixture_signal[0].t(),
            )
            sdr = sdr.cpu().numpy()
            sdr_baseline = sdr_baseline.cpu().numpy()
           
            sdr_i = sdr.mean() - sdr_baseline.mean()

            # Saving on a csv file
            row = {
               "audio_id": audio_id,
               "sdr": sdr.mean(),
               "sdr_i": sdr_i,
               "si-snr": -sisnr.item(),
               "si-snr_i": -sisnr_i.item(),
            }
            writer.writerow(row)

            # Metric Accumulation
            all_sdrs.append(sdr.mean())
            all_sdrs_i.append(sdr_i.mean())
            all_sisnrs.append(-sisnr.item())
            all_sisnrs_i.append(-sisnr_i.item())
      logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
      logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
      logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
      logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))   


In [4]:
def run():
    # Load hyperparameters file with command-line overrides
   hparams_file, run_opts, overrides = sb.parse_arguments(["hyperparams.yaml"])
   hparams_file = 'hyperparams.yaml'
#    run_opts = {"device": "cuda:0"}
   with open(hparams_file) as fin:
      hparams = load_hyperpyyaml(fin, overrides)
        
      # Load pretrained model if pretrained_separator is present in the yaml
   if "pretrained_separator" in hparams:
       # run_on_main(hparams["pretrained_separator"].collect_files)
       hparams["pretrained_separator"].load_collected(
            device=run_opts["device"]
       )
   # Brain class initialization
   separator = Separation(
        modules=hparams["modules"],
        run_opts={"device": "cuda"},
        hparams=hparams,
    )
   separator.get_metrics(audio_ids, out_target, out_est, out_mix)
run()

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
  0%|          | 0/3000 [00:00<?, ?it/s]

### Retrieves the worst, median and best separation audio mixtures

In [ ]:
import pandas as pd
df= pd.read_csv('results/test_results.csv')

# Get worst
def get_worst(column,df):
   return df.loc[df[column].idxmin()]

# Get best 
def get_best(column,df):
   return df.loc[df[column].idxmax()]
# Get Median
def get_median(column,df):
   df.loc[df[column]== df[column].median()]
   df.sort_values(by=column, inplace=True)
   return df[df[column] < df[column].median()].iloc[-1]


def print_audio_info(stat, df):
   print(f"status: {stat} | sdr: {df['sdr']} | sdr_i: {df['sdr_i']} | si-snr: {df['si-snr']} | si-snr_i: {df['si-snr_i']}")
status = ['Worst', 'Median', 'Best']

print_audio_info(status[0], get_worst('si-snr',df))
print_audio_info(status[0], get_median('si-snr',df))
print_audio_info(status[0], get_best('si-snr',df))
mean_values = df.loc[:, df.columns != 'audio_id'].mean()
mean_values

status: Worst | sdr: -1.8194904 | sdr_i: -2.4540033 | si-snr: -4.2891998291015625 | si-snr_i: -4.195562839508057
status: Worst | sdr: 18.402706 | sdr_i: 18.520067 | si-snr: 17.500259399414062 | si-snr_i: 17.701173782348633
status: Worst | sdr: 26.296406 | sdr_i: 26.069109 | si-snr: 26.019519805908203 | si-snr_i: 25.916349411010746


sdr         17.680850
sdr_i       17.500089
si-snr      16.967519
si-snr_i    16.967210
dtype: float64